주식 분석에 많이 사용되는 기술적분석으로 eda를 진행해 보도록 하겠습니다.<br>
기술적분석을 하는방법은 크게 2가지가 있는데 첫번째는 pandas와 numpy를 이용하여 직접하는 방법이고 두번째는 ta-lib을 사용하는 방법입니다.<br>
이중에서 ta-lib을 이용하여 eda를 진행해 보도록 하겠습니다.<br>
먼저, ta-lib을 사용하여 모든 피쳐를 구합니다.<br>
그리고 각 피쳐와 주식 변동을 비교하여 상대적으로 연관성이 높다고 판단되는 피쳐를 선택하는 식으로 진행합니다.<br>
아래는 랜덤으로 3개의 SecuritiesCode를 선택하여 진행하였습니다.<br>
ta를 하기전에 알아두셔야 할것은 대부분이 보조지표로 사용되며 실제 예측에서는 어디까지나 참고용으로 사용해야 합니다.

In [ ]:
!pip install --upgrade ta

In [ ]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pathlib import Path
import jpx_tokyo_market_prediction
import optuna
import ta
import copy
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from decimal import ROUND_HALF_UP, Decimal
mpl.style.use('seaborn')

valid_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=True)

print(valid_df.shape )

prices = valid_df

del valid_df

prices = prices.drop(["ExpectedDividend"],axis=1)

prices = prices.interpolate(method='linear',limit_direction='both')
prices.head()

In [ ]:
def date_sort(price):
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")
    price = price.sort_values(["SecuritiesCode", "Date"])

    price.set_index("Date", inplace=True)
    return price

In [ ]:
prices = date_sort(prices)
prices.info()

In [ ]:
prices = ta.add_all_ta_features(prices, "Open", "High", "Low", "Close", "Volume", fillna=False)

In [ ]:
prices.info()

ta의 add_all_ta_features 사용할 경우 ta의 함수로 구할수 있는 모든 수치들을 계산해줍니다

In [ ]:
stock_code_1 = 1333
stock_code_2 = 1793
stock_code_3 = 1407

위의 3개의 SecuritiesCode는 랜덤입니다.<br>

먼저 BB(Bollinger Bands)입니다<br>
20일 이동평균선을 사용하며 주가의 대략적인 이동을 보여줍니다.

In [ ]:
# BB

plt.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].Close, label='Close')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].volatility_bbh, label='High BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].volatility_bbl, label='Low BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].volatility_bbm, label='EMA BB')
plt.title('Bollinger Bands')
plt.legend()
plt.show()

In [ ]:
plt.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].Close, label='Close')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].volatility_bbh, label='High BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].volatility_bbl, label='Low BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].volatility_bbm, label='EMA BB')
plt.title('Bollinger Bands')
plt.legend()
plt.show()

In [ ]:
plt.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].Close, label='Close')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].volatility_bbh, label='High BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].volatility_bbl, label='Low BB')
plt.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].volatility_bbm, label='EMA BB')
plt.title('Bollinger Bands')
plt.legend()
plt.show()

3개의 주식에 적용시켜본 결과 close가 low보다 낮을 경우 단기적으로 주가가 하락하며<br>
High보다 높을 경우 단기적으로 상승하는것을 볼 수 있습니다.<br>
주가의 이동과 완전히 맞는것은 아니나 일시적인 폭락이나 폭등에 대해 어느정도 예측하게 해줍니다.

다음으로 adx(Average Directional Movement Index : 평균 방향 운동지수)입니다.<br>
추세지표의 일정으로 매매 추세를 보여줍니다.<br>
해당값이 높게 나타날경우 급등 혹은 급락이 일어날 가능성이있습니다.

In [ ]:
# adx 비교

fig = plt.figure(figsize=(6,6))
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'

ax1.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].trend_adx, color=color2)
ax2.set_ylabel('ADX', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

a

In [ ]:
# adx 비교

fig = plt.figure(figsize=(6,6))
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].trend_adx, color=color2)
ax2.set_ylabel('ADX', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

In [ ]:
# adx 비교

fig = plt.figure(figsize=(6,6))
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'

ax1.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].trend_adx, color=color2)
ax2.set_ylabel('ADX', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

3개의 그래프를 비교해 봤을 때 완전히 반영된다고는 할수없지만 현재의 추세를 바탕으로 앞으로 일어날 현상을 예측가능합니다.

3번째로 adx와 함께 많이 쓰이는 macd(Moving Average Convergence Divergence: 이동평균수렴·확산지수
)입니다.<br>
단기이동평균에서 중기이동편균뺀 값으로 실제 사용할때는 signal과 diff도 같이 사용됩니다.<br>
여기서는 macd만 보고 가겠습니다.

In [ ]:
# macd 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].trend_macd, color=color2)
ax2.set_ylabel('MACD', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

In [ ]:
# macd 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].trend_macd, color=color2)
ax2.set_ylabel('MACD', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

In [ ]:
# macd 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot() 
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].trend_macd, color=color2)
ax2.set_ylabel('MACD', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

1번의 경우 반영이 잘 되지 않았으나 2,3 번의 경우 반영이 된 것을 확인할 수 있습니다.<br>
조금 더 정확하게 사용하려면 macd를 close와 분리하고 signal과 diff를 추가하는 것을 권장합니다.

마지막으로 rsi(relative strength index: 상대강도지수)입니다.<br>
상승압력과 하강압력의 상대적인 값이며 변화량이 크면 과매도로 작으면 과매수로 판단합니다.<br>
보통 과매수와 과매도는 70이상, 30이하로 구분됩니다.

In [ ]:
# momentum_rsi 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_1][20:].momentum_rsi, color=color2)
ax2.set_ylabel('momentum_rsi', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

In [ ]:
# momentum_rsi 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot()
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_2][20:].momentum_rsi, color=color2)
ax2.set_ylabel('momentum_rsi', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

In [ ]:
# momentum_rsi 비교

fig = plt.figure(figsize=(6,6)) 
fig.set_facecolor('white')
ax1 = fig.add_subplot() 
color1 = 'royalblue'
color2 = 'lightcoral'
 
ax1.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].Close, color=color1)
ax1.set_xlabel('x')
ax1.set_ylabel('Close', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
ax2 = ax1.twinx()
ax2.plot(prices[prices["SecuritiesCode"] == stock_code_3][20:].momentum_rsi, color=color2)
ax2.set_ylabel('momentum_rsi', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
 
plt.show()

위의 표로 보아 급락 직전 혹은 도중에 rsi 값이 30이하로 떨어지는 형태를 보입니다.<br>
표1에서만 관측이 되었는데 급등 직전 혹은 도중에 70이상으로 오르는 것도 확인되었습니다.

이상으로 총 4가지 지표 BB, ADX, MACD, RSI에 대하여 살펴보았습니다.<br>
약간의 수정으로 다른 지표도 볼수 있으니 필요하신분은 잘 활용하셨으면 합니다.<br>
다음에는 수치 직접계산과 feature engineering에 대해 다뤄보겠습니다.<br>
https://www.kaggle.com/code/jihoryu1/feature-engineering-test